In [1]:
!pip install..
!pip install findspark

ERROR: unknown command "install.." - maybe you meant "install"

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [11]:
!pip show pyjedai


Name: pyjedai
Version: 0.0.10
Summary: An open-source library that builds powerful end-to-end Entity Resolution workflows.
Home-page: 
Author: 
Author-email: Konstantinos Nikoletos <nikoletos.kon@gmail.com>, Jakub Maciejewski <jacobb.maciejewski@gmail.com>, George Papadakis <gpapadis84@gmail.com>
License: Apache Software License 2.0
Location: /home/vefa/Desktop/pyJedAI-Dev/src
Requires: faiss-cpu, gensim, matplotlib, matplotlib-inline, networkx, nltk, numpy, optuna, ordered-set, pandas, pandas-profiling, pandocfilters, plotly, py-stringmatching, pyspark, PyYAML, rdflib, rdfpandas, regex, scipy, seaborn, sentence-transformers, strsim, strsimpy, tomli, tqdm, transformers, valentine
Required-by: 


In [12]:
import os
import sys
import pandas as pd
import networkx
from networkx import draw, Graph

import findspark

findspark.init()

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"


In [13]:
from pyspark import SparkConf, SparkContext


from pyspark import SparkConf, SparkContext


# Configure Spark to use more driver memory and executor memory
conf = SparkConf().setAppName('PyJedAI').setMaster('local[*]')
conf.set("spark.executor.memory", "4g")
conf.set("spark.driver.memory","2g")
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")


# Create a SparkContext with the configured properties
sc = SparkContext(conf=conf)


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=PyJedAI, master=local[*]) created by __init__ at /tmp/ipykernel_27073/4027621325.py:15 

In [5]:
from pyjedai.datamodel import Data

d1 = pd.read_csv("./../data/der/cora/cora.csv", sep='|')
gt = pd.read_csv("./../data/der/cora/cora_gt.csv", sep='|', header=None)
attr = ['Entity Id', 'author', 'title']

In [6]:
data = Data(
    dataset_1=d1,
    id_column_name_1='Entity Id',
    ground_truth=gt,
    attributes_1=attr
)

In [7]:
from pyjedai.block_building import (
    StandardBlocking,
    QGramsBlocking,
    SuffixArraysBlocking,
    ExtendedSuffixArraysBlocking,
    ExtendedQGramsBlocking
)

from pyjedai.parallel.block_building_spark import (
    StandardBlockingSpark,
    QGramsBlockingSpark,
    ExtendedQGramsBlockingSpark,
    SuffixArraysBlockingSpark,
    ExtendedSuffixArraysBlockingSpark,
)


In [8]:
bb = StandardBlocking()
blocks = bb.build_blocks(data)

Standard Blocking:   0%|          | 0/1295 [00:00<?, ?it/s]

In [9]:
_ = bb.evaluate(blocks)

***************************************************************************************************************************
                                         Μethod:  Standard Blocking
***************************************************************************************************************************
Method name: Standard Blocking
Parameters: 
Runtime: 0.1301 seconds
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Performance:
	Precision:      0.64% 
	Recall:        99.99%
	F1-score:       1.27%
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [10]:
bb = StandardBlockingSpark(sc)

blocks = bb.build_blocks(data)

Standard Blocking:   0%|          | 0/5 [00:00<?, ?it/s]

0


In [9]:
_ = bb.evaluate(dict(blocks.collect()),with_classification_report=True)


***************************************************************************************************************************
                                         Μethod:  Standard Blocking
***************************************************************************************************************************
Method name: Standard Blocking
Parameters: 
Runtime: 1.2066 seconds
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Performance:
	Precision:      0.64% 
	Recall:        99.99%
	F1-score:       1.27%
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Classification report:
	True positives: 17183
	False positives: 2663261
	True negatives: 820680
	False negatives: 1
	Total comparisons: 2680444
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [ ]:
# sc.stop()